In [88]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams, cm
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [89]:
H = 0.25
L = 1

alpha = 207*10**(-6) #Volume expansion coefficient of water @ 20 C
k = 0.6     #Thermal conductivity of water @ 20 C

#ReC = 1100.65 #Critical Rayleigh number for one free and one rigid boundary condition
ReC = 1708     #Critical Rayleigh number for two rigid body boundary condition
g = 9.81
D = 1.43E-7  #Thermal diffusivity of water
c = k/D
rho_i = 1000   #Density of water @ 20 C

L = 1
H = 0.001

nxy = 101
nt = 6000

dxy = H/(nxy-1)

x = np.linspace(0,L,nxy)
y = np.linspace(0,H,nxy)
ya = np.ones((nxy,nxy))
for i in range(nxy):
    ya[:,i] = y
#Change to Kelivin?
To = 20
vo = 2.414*10**(-5) * 10**(247.8/(To + 273 - 140)) / rho_i  #initial kinematic viscosity of water
delT = (ReC*vo*1.43*10**(-7))/(alpha*g*H**3)
Ti = np.ones((nxy, nxy))*(To + (H-ya)/H*(70-To))
#Ti[-1,:] = To + 50

rho = np.ones((nxy,nxy), dtype=float)*rho_i
rho[0,:] = rho_i*(1-alpha*(Ti[0,:] - To))

v = np.ones((nxy,nxy))*2.414*10**(-5) * 10**(247.8/(Ti + 273 - 140)) / rho  #kinematic viscosity of water

pi = np.zeros((nxy,nxy), dtype=float)
pi[:,:] = rho[:,:] * (H-ya[:,:]) * g

ui = np.zeros((nxy,nxy), dtype=float)

print('delT =',delT, 'Tl =',To+delT)
print(v)
#print('x =', x, 'y =', y)

delT = 120.92877677020469 Tl = 140.9287767702047
[[  4.05457457e-07   4.05457457e-07   4.05457457e-07 ...,   4.05457457e-07
    4.05457457e-07   4.05457457e-07]
 [  4.04055450e-07   4.04055450e-07   4.04055450e-07 ...,   4.04055450e-07
    4.04055450e-07   4.04055450e-07]
 [  4.06883369e-07   4.06883369e-07   4.06883369e-07 ...,   4.06883369e-07
    4.06883369e-07   4.06883369e-07]
 ..., 
 [  9.81359599e-07   9.81359599e-07   9.81359599e-07 ...,   9.81359599e-07
    9.81359599e-07   9.81359599e-07]
 [  9.93275020e-07   9.93275020e-07   9.93275020e-07 ...,   9.93275020e-07
    9.93275020e-07   9.93275020e-07]
 [  1.00541442e-06   1.00541442e-06   1.00541442e-06 ...,   1.00541442e-06
    1.00541442e-06   1.00541442e-06]]


In [90]:
# stability
sigma = 0.00005
#dt = sigma*min(dxy,dxy)**2/D
dt = 0.001

print(dt)

0.001


In [91]:
print(dxy)

1e-05


In [92]:
def constructMatrix(nxy):
    """ Generate implicit matrix for 2D heat equation with Dirichlet in bottom and right and Neumann in top and left
        Assumes dx = dy
    
    Parameters:
    ----------
    nx   : int
        number of discretization points in x
    ny   : int
        number of discretization points in y
    sigma: float
        alpha*dt/dx
        
    Returns:
    -------
    A: 2D array of floats
        Matrix of implicit 2D heat equation
    """
    
    A = np.zeros(((nxy)*(nxy-2),(nxy)*(nxy-2)))
    
    row_number = 0 # row counter
    for j in range(1,nxy-1):
        for i in range(nxy):
                
            if j==1: # Bottom boundary (Dirichlet)
                A[row_number,row_number] = -4 # Set diagonal
                A[row_number,row_number+1] = 1      # fetch i+1
                A[row_number,row_number-1] = 1      # fetch i-1
                A[row_number,row_number+nxy] = 1   # fetch j+1
                
            elif j==nxy-2: # Top boundary (Neumann)
                A[row_number,row_number] = -4 # Set diagonal
                A[row_number,row_number+1] = 1      # fetch i+1
                A[row_number,row_number-1] = 1      # fetch i-1
                A[row_number,row_number-(nxy)] = 1 # fetch j-1
                
            # Interior points
            else:
                A[row_number,row_number] = -4 # Set diagonal
                A[row_number,row_number+1] = 1      # fetch i+1
                A[row_number,row_number-1] = 1      # fetch i-1
                A[row_number,row_number+nxy] = 1   # fetch j+1
                A[row_number,row_number-(nxy)] = 1 # fetch j-1
                
            row_number += 1 # Jump to next row of the matrix!
    
    return A           

In [93]:
def generateRHS(nxy, dxy, dt, u, rho_i):
    """ Generates right-hand side for 2D implicit heat equation with Dirichlet in bottom and left and Neumann in top and right
        Assumes dx=dy, Neumann BCs = 0, and constant Dirichlet BCs
        
        Paramenters:
        -----------
        nx   : int
            number of discretization points in x
        ny   : int
            number of discretization points in y
        sigma: float
            alpha*dt/dx
        T    : array of float
            Temperature in current time step
        T_bc : float
            Temperature in Dirichlet BC
        
        Returns:
        -------
        RHS  : array of float
            Right hand side of 2D implicit heat equation
    """
    RHS = np.zeros((nxy)*(nxy-2))
    
    row_number = 0 # row counter
    for j in range(1,nxy-1):
        for i in range(nxy):
              
            if j==1: # Bottom boundary (Dirichlet)
                RHS[row_number] = 0
                
            elif j==nxy-2: # Top boundary (Neumann)
                RHS[row_number] = 0
                
            # Interior points
            else:
                RHS[row_number] = (rho_i*dxy/(2*dt))*(u[2:,1:-1]-u[:-2,1:-1] + u[1:-1,2:] - u[1:-1,:-2]) 
                
            row_number += 1 # Jump to next row!
    
    return RHS

In [94]:
def map_1Dto2D(nxy, u_1D):
    """ Takes temperatures of solution of linear system, stored in 1D, 
    and puts them in a 2D array with the BCs
    Valid for constant Dirichlet bottom and left, and Neumann with zero 
    flux top and right
        
    Parameters:
    ----------
        nx  : int
            number of nodes in x direction
        ny  : int
            number of nodes in y direction
        T_1D: array of floats
            solution of linear system
        T_bc: float
            Dirichlet BC
            
    Returns:
    -------
        T: 2D array of float
            Temperature stored in 2D array with BCs
    """
    u = np.zeros((nxy,nxy))
    
    row_number = 0
    for j in range(1,nxy-1):
        for i in range(nxy):
            u[j,i] = u_1D[row_number]
            row_number += 1
    # Dirichlet BC
    u[0,:] = 0
    u[-1,:] = 0
    
    return u

In [95]:
def btcs_2D(u, A, nt, nxy, dxy, dt, rho_i):
    """ Advances diffusion equation in time with backward Euler
   
    Parameters:
    ----------
    T: 2D array of float
        initial temperature profile
    A: 2D array of float
        Matrix with discretized diffusion equation
    nt: int
        number of time steps
    sigma: float
        alpha*dt/dx^2
    T_bc : float 
        Dirichlet BC temperature
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
    dt   : float
        Time step size
        
    Returns:
    -------
    T: 2D array of floats
        temperature profile after nt time steps
    """
    
    
    for t in range(nt):
        un = u.copy()
        b = generateRHS(nxy, dxy, dt, u, rho_i)
        # Use numpy.linalg.solve
        p_interior = solve(A,b)
        p = map_1Dto2D(nxy, p_interior)
        
    return p

In [96]:
def ftcs(T, To, u, p, rho, rho_i, alpha, v, k, c, g, ya, H, delT, nt, dt, dxy, nxy):
    
    for n in range(nt):

        un = u.copy()
        u_star = un.copy()
        Tn = T.copy()
        pn = p.copy()
        vn = v.copy()
        
        u_star[1:-1,1:-1] = un[1:-1,1:-1] + dt*(-(u[1:-1,1:-1]/(2*dxy) * (u[2:,1:-1] - u[:-2,1:-1] + u[1:-1,2:] - u[1:-1,:-2])\
                            + v[1:-1,1:-1]/dxy**2 * (u[2:,1:-1] + u[:-2,1:-1] + u[1:-1,2:] + u[1:-1,:-2] - 4*u[1:-1,1:-1])\
                                                 + rho[1:-1,1:-1]/rho_i * g))
        
        u_star[-1,:] = 0 #Top Boundary
        u_star[0,:] = 0   #Bottom Boundary
        
        #p[1:-1,1:-1] = pn[1:-1,1:-1] + 1/(2*dt*dxy) * (u_star[2:,1:-1] - u_star[:-2,1:-1] + u_star[1:-1,2:] - u_star[1:-1,:-2])\
        
        A = constructMatrix(nxy)
        p = btcs_2D(u_star, A, nt, nxy, dxy, dt, rho_i)
        
        u[1:-1,1:-1] = u_star[1:-1,1:-1] - dt/(2*dxy)*(p[2:,1:-1] - p[:-2,1:-1] + p[1:-1,2:] - p[1:-1,:-2])
        
        u[-1,:] = 0 #Top Boundary
        u[0,:] = 0   #Bottom Boundary
        
        T[1:-1,1:-1] = T[1:-1,1:-1] + dt*(-u[1:-1,1:-1]/(2*dxy) * (T[2:,1:-1]-T[:-2,1:-1]+T[1:-1,2:]-T[1:-1,:-2])\
                                         + k/(rho[1:-1,1:-1]*c*dxy**2) * (T[2:,1:-1] + T[:-2,1:-1] + T[1:-1,2:] + T[1:-1,:-2]\
                                                                         -4*T[1:-1,1:-1]))
    
        T[0,:] = To + 50 #Bottom Boundary
        T[-1,:] = To   #top Boundary
        
        rho[1:-1,1:-1] = rho_i*(1- alpha*(T[1:-1,1:-1] - To))
        
        v[1:-1,1:-1] = 2.414*10**(-5) * 10**(247.8/(T[1:-1,1:-1] + 273 - 140)) / rho[1:-1,1:-1]
        
        
        #del u = 0
        #u[1:,1:] = 0.5*(u[:-1,1:] + u[1:,:-1])
        
        

                                           
    return u, T

In [97]:
u, T = ftcs(Ti,To,ui,pi,rho,rho_i,alpha,v,k,c,g,ya,H,delT,nt,dt,dxy,nxy)

IndexError: index 9999 is out of bounds for axis 1 with size 9999

In [ ]:
plt.figure(figsize=(8,5))
plt.contourf(x,y,T,20,cmap=cm.viridis)
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.colorbar();


In [ ]:
print(T)

In [ ]:
print(ui)

In [ ]:
print(pi)